# Daniel's Pre-Processing


Daniel's Branch Created


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_json('../Dataset/train.json')
test_df = pd.read_json('../Dataset/test.json')

In [3]:
train_df.head()

,premise,hypothesis,label
0,Pluto rotates once on its axis every 6.39 Eart...,Earth rotates on its axis once times in one day.,neutral
1,---Glenn =====================================...,Earth rotates on its axis once times in one day.,entails
2,geysers - periodic gush of hot water at the su...,The surface of the sun is much hotter than alm...,neutral
3,Facts: Liquid water droplets can be changed in...,Evaporation is responsible for changing liquid...,entails
4,"By comparison, the earth rotates on its axis o...",Earth rotates on its axis once times in one day.,entails


In [4]:
train_df.tail()

,premise,hypothesis,label
23083,"which is not only the motion of our bodies, bu...",Work is done only if a force is exerted in the...,neutral
23084,"The Red Star, that celestial curse whose eccen...",Red-shift refers to a shift toward red in the ...,neutral
23085,The lines in the spectrum of a luminous body s...,Red-shift refers to a shift toward red in the ...,entails
23086,The radial velocity of a star away from or tow...,Red-shift refers to a shift toward red in the ...,entails
23087,This expansion causes the light from distant s...,Red-shift refers to a shift toward red in the ...,entails


# Data Pre-Processing


In [5]:
import nltk
if not nltk.download('punkt'):
    nltk.download('punkt')
    nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

if not nltk.download('stopwords'):
    nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DCCN9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DCCN9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# These are just common English contractions. We used it in Lab 5 before!
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [7]:
# Stemmer and Lemmatizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize

if not nltk.download('wordnet'):
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DCCN9\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Helper function for lemmatization with POS tagging
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

In [9]:
# Allowed POS tags for filtering (example: nouns, verbs, adjectives, adverbs)
allowed_pos_tags = {'NN', 'NNS', 'NNP', 'NNPS',   # Nouns
                    'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',  # Verbs
                    'JJ', 'JJR', 'JJS',           # Adjectives
                    'RB', 'RBR', 'RBS'}           # Adverbs

In [10]:
# Filter the desired POS tags
def filter_tokens_by_pos(tokens):
    tagged_tokens = pos_tag(tokens)
    filtered = [word for word, tag in tagged_tokens if tag in allowed_pos_tags]
    return filtered

In [11]:
# Config
min_word_length = 3
max_word_length = 60 

In [14]:
def clean_dataset(dataset, min_word_length=3, max_word_length=50, method='none', pos_filter=False, stop_w=False):
    cleaned_premises = []
    cleaned_hypotheses = []
    cleaned_labels = []

    for index, row in dataset.iterrows():
        premise = row['premise']
        hypothesis = row['hypothesis']
        label = row['label']

        # Lowercase
        premise = premise.lower()
        hypothesis = hypothesis.lower()

        # Expand contractions
        for contraction, full_form in contraction_dict.items():
            premise = premise.replace(contraction, full_form)
            hypothesis = hypothesis.replace(contraction, full_form)

        # Remove punctuation/special chars
        premise = re.sub(r'[^\w\s]', '', premise)
        hypothesis = re.sub(r'[^\w\s]', '', hypothesis)

        # Normalize whitespace
        premise = re.sub(r'\s+', ' ', premise).strip()
        hypothesis = re.sub(r'\s+', ' ', hypothesis).strip()

        # Tokenization
        premise_tokens = word_tokenize(premise)
        hypothesis_tokens = word_tokenize(hypothesis)

        # Stemming/Lemmatization
        if method == 'stem':
            stemmer = PorterStemmer()
            premise_tokens = [stemmer.stem(word) for word in premise_tokens]
            hypothesis_tokens = [stemmer.stem(word) for word in hypothesis_tokens]
        elif method == 'lemmatize':
            lemmatizer = WordNetLemmatizer()
            premise_pos_tags = pos_tag(premise_tokens)
            hypothesis_pos_tags = pos_tag(hypothesis_tokens)
            premise_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in premise_pos_tags]
            hypothesis_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in hypothesis_pos_tags]

        # POS Filtering
        if pos_filter:
            premise_tokens = filter_tokens_by_pos(premise_tokens)
            hypothesis_tokens = filter_tokens_by_pos(hypothesis_tokens)

        # Remove stop words
        if stop_w:
            premise_tokens = [word for word in premise_tokens if word not in stop_words]
            hypothesis_tokens = [word for word in hypothesis_tokens if word not in stop_words]

        # Now check token length AFTER cleaning
        if (min_word_length <= len(premise_tokens) <= max_word_length and
            min_word_length <= len(hypothesis_tokens) <= max_word_length):
            cleaned_premises.append(premise_tokens)
            cleaned_hypotheses.append(hypothesis_tokens)
            cleaned_labels.append(label)
        # else: skip row

    # Build DataFrame from all cleaned token lists
    new_dataset = pd.DataFrame({
        'premise': cleaned_premises,
        'hypothesis': cleaned_hypotheses,
        'label': cleaned_labels
    })

    return new_dataset


In [15]:
clean_train_dataset = clean_dataset(train_df, stop_w=True)

In [19]:
clean_train_dataset.head()

,premise,hypothesis,label
0,"[pluto, rotates, axis, every, 639, earth, days]","[earth, rotates, axis, times, one, day]",neutral
1,"[glenn, per, day, earth, rotates, axis]","[earth, rotates, axis, times, one, day]",entails
2,"[geysers, periodic, gush, hot, water, surface,...","[surface, sun, much, hotter, almost, anything,...",neutral
3,"[facts, liquid, water, droplets, changed, invi...","[evaporation, responsible, changing, liquid, w...",entails
4,"[comparison, earth, rotates, axis, per, day, r...","[earth, rotates, axis, times, one, day]",entails


## Build dictionary of words and word2id


In [ ]:
print(clean_train_dataset['premise'])

In [ ]:
# Create a unique word list from the cleaned dataset
unique_words = set()
for sentence in clean_train_dataset['premise']:
    for word in sentence:
        unique_words.add(word)

print(f"Count of unique words: {len(unique_words)}")
#No lemmatization or stemming count = 22555
#Lemmatization count = 18986
#Stemming count = 15954


Count of unique words: 21790


In [42]:
unique_words_list = sorted(list(unique_words))

In [46]:
# No. of words with each method.

word2id = {w:i for i,w in enumerate(unique_words_list)}
id2word = {i:w for i,w in enumerate(unique_words_list)}


In [47]:
word2id

{'0': 0,
 '000': 1,
 '000010m': 2,
 '0001': 3,
 '000338324205530': 4,
 '004': 5,
 '0055': 6,
 '0075ml': 7,
 '01': 8,
 '010': 9,
 '0104': 10,
 '010508': 11,
 '012': 12,
 '014': 13,
 '0147this': 14,
 '016': 15,
 '02': 16,
 '02cm2': 17,
 '03': 18,
 '04': 19,
 '0405060708': 20,
 '04622quot': 21,
 '05': 22,
 '05c': 23,
 '05v': 24,
 '06': 25,
 '060721': 26,
 '07': 27,
 '070b': 28,
 '075': 29,
 '092': 30,
 '093': 31,
 '1': 32,
 '10': 33,
 '100': 34,
 '1000': 35,
 '100000': 36,
 '10000000': 37,
 '1000000000000000': 38,
 '100000th': 39,
 '10001500': 40,
 '10002000': 41,
 '1001': 42,
 '10020': 43,
 '100fold': 44,
 '100ml': 45,
 '100um': 46,
 '101': 47,
 '101325': 48,
 '102': 49,
 '1020': 50,
 '1023': 51,
 '103': 52,
 '1033': 53,
 '104': 54,
 '1040': 55,
 '105': 56,
 '106': 57,
 '1069': 58,
 '107': 59,
 '107607676383': 60,
 '109': 61,
 '1093': 62,
 '10e4': 63,
 '11': 64,
 '110': 65,
 '1104': 66,
 '1107': 67,
 '111': 68,
 '1117': 69,
 '113': 70,
 '113000': 71,
 '11313': 72,
 '115': 73,
 '1152': 74

# Build Embedding Model using Gensim and FastText


In [48]:
from gensim.models import FastText

In [49]:
# Hyperparameters
batch_size = 1024
learn_rate = 0.01
embedding_size = 200
no_of_epochs = 8
window_size = 8
vocab_size = len(unique_words)


In [50]:
clean_t_dataset = clean_train_dataset['premise'] + clean_train_dataset['hypothesis']

In [51]:
fast_text_model = FastText(clean_t_dataset, 
                           vector_size=embedding_size,
                           window=window_size,
                           sg=1,
                           epochs=no_of_epochs
                           )

In [55]:
fast_text_model.wv.most_similar('saturn', topn=10)

[('saturns', 0.9604904055595398),
 ('1980', 0.8468125462532043),
 ('voyager', 0.8449878096580505),
 ('1979', 0.8226709365844727),
 ('neptune', 0.8205943703651428),
 ('spacecraft', 0.819324254989624),
 ('jupiter', 0.7984594106674194),
 ('1989', 0.7962067723274231),
 ('1981', 0.7874003052711487),
 ('1980s', 0.7858580946922302)]

# Transformer Model Implementation
